<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li></ul></div>

## Подготовка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.utils import shuffle

data = pd.read_csv('/datasets/Churn.csv')

# Просмотр информации о датафрейме
display(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


None

In [2]:
# Просмотр пропущенных значений
print(data.isnull().sum())

RowNumber            0
CustomerId           0
Surname              0
CreditScore          0
Geography            0
Gender               0
Age                  0
Tenure             909
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
dtype: int64


In [3]:
# Просмотр первых 10 строк таблицы (датафрейма)
display(data.head(10))

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


In [4]:
# Просмотр случайной выборки из таблицы (датафрейма)
display(data.sample())

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
3423,3424,15660147,Dore,493,Spain,Male,32,NaN,46161.18,1,1,1,79577.4,0


In [5]:
# Просмотр последних 10 строк таблицы (датафрейма)
display(data.tail(10))

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
9990,9991,15798964,Nkemakonam,714,Germany,Male,33,3.0,35016.60,1,1,0,53667.08,0
9991,9992,15769959,Ajuluchukwu,597,France,Female,53,4.0,88381.21,1,1,0,69384.71,1
9992,9993,15657105,Chukwualuka,726,Spain,Male,36,2.0,0.00,1,1,0,195192.40,0
9993,9994,15569266,Rahman,644,France,Male,28,7.0,155060.41,1,1,0,29179.52,0
9994,9995,15719294,Wood,800,France,Female,29,2.0,0.00,2,0,0,167773.55,0
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1
9999,10000,15628319,Walker,792,France,Female,28,NaN,130142.79,1,1,0,38190.78,0


In [6]:
# Просмотр максимальных и минимальных, а также уникальных значений в некоторых столбцах таблицы
# Максимальный и минимальный кредитного рейтинга 
print('Кредитный рейтинг (CreditScore), min/max:', data['CreditScore'].min(), '/', data['CreditScore'].max())

# Уникальные значения пола
print('Пол (Gender), уникальные значения:', sorted(data['Gender'].unique()))

# Уникальные значения возраста
print('Возраст (Age), уникальные значения:', sorted(data['Age'].unique()))
# Максимальный и минимальный возраст 
print('Возраст (Age), min/max:', data['Age'].min(), '/', data['Age'].max())

# Уникальные значения страны проживания
print('Страна проживания (Geography), уникальные значения:', sorted(data['Geography'].unique()))

# Максимальное и минимальное количество лет сколько человек является клиентом банка 
print('Сколько лет человек является клиентом банка (Tenure), min/max:', data['Tenure'].min(), '/', data['Tenure'].max())
# Уникальные значения количества лет сколько человек является клиентом банка 
print('Сколько лет человек является клиентом банка (Tenure), уникальные значения:', sorted(data['Tenure'].unique()))

# Максимальный и минимальный баланс на счёте 
print('Баланс на счёте (Balance), min/max:', data['Balance'].min(), '/', data['Balance'].max())

# Уникальные значения количества продуктов банка, используемых клиентом
print('Количество продуктов банка, используемых клиентом (NumOfProducts), уникальные значения:', sorted(data['NumOfProducts'].unique()))

# Уникальные значения признака наличия кредитной карты
print('Наличие кредитной карты (HasCrCard), уникальные значения:', sorted(data['HasCrCard'].unique()))

# Уникальные значения признака активности клиента
print('Активность клиента (IsActiveMember), уникальные значения:', sorted(data['IsActiveMember'].unique()))

# Максимальная и минимальная предполагаемая зарплата
print('Предполагаемая зарплата (EstimatedSalary), min/max:', data['EstimatedSalary'].min(), '/', data['EstimatedSalary'].max())

# Уникальные значения признака факт ухода клиента
print('Факт ухода клиента (Exited), уникальные значения:', sorted(data['Exited'].unique()))

# Проверка количества дубликатов в таблице (датафрейме)
print('Количество явных дубликатов:', data.duplicated().sum())

# Проверка на неявные дубликаты
# если дубликатов нет, то количство уникальных значений совпадает с количеством записей в столбце
print(f'Количество записей в RowNumber: {data["RowNumber"].count()}')
print(f'Количество уникальных значений RowNumber: {len(sorted(data["RowNumber"].unique()))}')
print(f'Количество записей в CustomerId: {data["CustomerId"].count()}')
print(f'Количество уникальных значений CustomerId: {data["CustomerId"].unique().shape[0]}')

Кредитный рейтинг (CreditScore), min/max: 350 / 850
Пол (Gender), уникальные значения: ['Female', 'Male']
Возраст (Age), уникальные значения: [18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 88, 92]
Возраст (Age), min/max: 18 / 92
Страна проживания (Geography), уникальные значения: ['France', 'Germany', 'Spain']
Сколько лет человек является клиентом банка (Tenure), min/max: 0.0 / 10.0
Сколько лет человек является клиентом банка (Tenure), уникальные значения: [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, nan]
Баланс на счёте (Balance), min/max: 0.0 / 250898.09
Количество продуктов банка, используемых клиентом (NumOfProducts), уникальные значения: [1, 2, 3, 4]
Наличие кредитной карты (HasCrCard), уникальные значения: [0, 1]
Активность клиента (IsActiveMembe

In [7]:
# Удалим столбцы, которые не понадобятся для обучения
data = data.drop(['RowNumber','CustomerId', 'Surname'], axis=1)
display(data.sample())

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
6805,614,France,Female,35,1.0,0.0,2,1,1,3342.62,0


In [8]:
#     RowNumber — индекс строки в данных
#     CustomerId — уникальный идентификатор клиента
#     Surname — фамилия
#     CreditScore — кредитный рейтинг
#     Geography — страна проживания
#     Gender — пол
#     Age — возраст
#     Tenure — сколько лет человек является клиентом банка
#     Balance — баланс на счёте
#     NumOfProducts — количество продуктов банка, используемых клиентом
#     HasCrCard — наличие кредитной карты
#     IsActiveMember — активность клиента
#     EstimatedSalary — предполагаемая зарплата

# Заменим пропущенные значения медианой
tenure_median = data['Tenure'].median()
data['Tenure'] = data['Tenure'].fillna(tenure_median)

# Создадим датафрейм с защитой от дамми-ловушки
data_dummy = pd.get_dummies(data, drop_first=True)
target = data_dummy['Exited']
features = data_dummy.drop(['Exited'] , axis=1)

# Обучающая выборка
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size = 0.4, random_state = 12345)

# Валидационная и тестовая выборки
features_valid, features_test, target_valid, target_test = train_test_split(
    features_test, target_test, test_size = 0.5, random_state = 12345)

# Промасштабируем количественно-численные признаки
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
scaler = StandardScaler()
scaler.fit(features_train[numeric])

features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

print(f'Размер обучающей выборки: {features_train.shape}')
print(f'Размер валидационной выборки: {features_valid.shape}')
print(f'Размер тестовой выборки: {features_test.shape}')

Размер обучающей выборки: (6000, 11)
Размер валидационной выборки: (2000, 11)
Размер тестовой выборки: (2000, 11)


/tmp/ipykernel_107/1399591368.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_train[numeric] = scaler.transform(features_train[numeric])
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


<b>Вывод:</b>

Просмотрены и подготовлены данные к исследованию. Сформированы выборки: обучающая, валидационная и тестовая.

## Исследование задачи

In [9]:
# Определение лучшей глубины решающего дерева
best_depth_dt = 0
best_f1_score_dt = 0

for depth in range(1,30):
    model_dt = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_dt.fit(features_train, target_train)
    predicted_valid_dt = model_dt.predict(features_valid)
    score_dt = f1_score(target_valid, predicted_valid_dt)
    if score_dt > best_f1_score_dt:
        best_depth_dt = depth
        best_f1_score_dt = score_dt

print(f'Лучшая глубина решающего дерева: {best_depth_dt}, f1-score: {best_f1_score_dt:.3f}')

Лучшая глубина решающего дерева: 6, f1-score: 0.570


In [10]:
# Определение лучшего количества деревьев случайного леса
best_est = 0
best_f1_score_rf = 0

for est in range(1, 50):
    model_rf = RandomForestClassifier(random_state=12345, n_estimators=est)
    model_rf.fit(features_train, target_train)
    predicted_valid_rf = model_rf.predict(features_valid)
    score_rf = f1_score(target_valid, predicted_valid_rf)
    if score_rf > best_f1_score_rf:
        best_est = est
        best_f1_score_rf = score_rf

print(f'Лучшее количество деревьев случайного леса: {best_est}, f1-score: {best_f1_score_rf:.3f}')

Лучшее количество деревьев случайного леса: 23, f1-score: 0.588


In [11]:
# Определение лучшей глубины случайного леса
best_depth_rf = 0
best_f1_score_rf = 0

for depth in range(1,50):
    model_rf = RandomForestClassifier(random_state=12345, n_estimators=49, max_depth=depth)
    model_rf.fit(features_train, target_train)
    predicted_valid_rf = model_rf.predict(features_valid)
    score_rf = f1_score(target_valid, predicted_valid_rf)
    if score_rf > best_f1_score_rf:
        best_depth_rf = depth
        best_f1_score_rf = score_rf

print(f'Лучшая глубина случайного леса: {best_depth_rf}, f1-score: {best_f1_score_rf:.3f}')

Лучшая глубина случайного леса: 19, f1-score: 0.585


Гиперпараметр max_depth вместе с n_estimators даёт f1-score ниже, чем при использовании только n_estimators. Будем использовать только n_estimators.

In [12]:
# Создание функции проверки модели на адекватность, f1 и ROC
def сheck_adequacy_f1_roc(name, model, features_valid, target_valid, features_train, target_train):
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, predicted_valid)
    precision = precision_score(target_valid, predicted_valid, zero_division=0)
    recall = recall_score(target_valid, predicted_valid, zero_division=0)
    f1 = f1_score(target_valid, predicted_valid, zero_division=0)

    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

    print(f'\n{name}')
    print(f'Доля правильных ответов: {accuracy}, precision: {precision:.3f}, recall: {recall:.3f}, f1: {f1:.3f}, roc_auc: {auc_roc:.3f}')

# Создание и обучение модели Решающее дерево
model_dt = DecisionTreeClassifier(random_state=12345, max_depth=6)
сheck_adequacy_f1_roc('Решающее дерево', model_dt, features_valid, target_valid, features_train, target_train)

# Создание и обучение модели Случайный лес
model_rf = RandomForestClassifier(random_state=12345, n_estimators=23)
сheck_adequacy_f1_roc('Случайный лес', model_rf, features_valid, target_valid, features_train, target_train)


Решающее дерево
Доля правильных ответов: 0.858, precision: 0.777, recall: 0.450, f1: 0.570, roc_auc: 0.816

Случайный лес
Доля правильных ответов: 0.858, precision: 0.746, recall: 0.486, f1: 0.588, roc_auc: 0.833

Логистическая регрессия
Доля правильных ответов: 0.802, precision: 0.563, recall: 0.234, f1: 0.331, roc_auc: 0.759


<b>Вывод:</b>

По полученному результату видно, что f1 и ROC-кривая лучше у случайного леса.

## Борьба с дисбалансом

In [13]:
# Рассмотрим взвешивание классов
# Определение лучшей глубины решающего дерева
best_depth_dt = 0
best_f1_score_dt = 0

for depth in range(1,30):
    model_dt = DecisionTreeClassifier(random_state=12345, max_depth=depth, class_weight='balanced')
    model_dt.fit(features_train, target_train)
    predicted_valid_dt = model_dt.predict(features_valid)
    score_dt = f1_score(target_valid, predicted_valid_dt)
    if score_dt > best_f1_score_dt:
        best_depth_dt = depth
        best_f1_score_dt = score_dt

print(f'Лучшая глубина решающего дерева: {best_depth_dt}, f1-score: {best_f1_score_dt:.3f}')

Лучшая глубина решающего дерева: 5, f1-score: 0.596


In [14]:
# Определение лучшего количества деревьев случайного леса
best_est = 0
best_f1_score_rf = 0

for est in range(1, 50):
    model_rf = RandomForestClassifier(random_state=12345, n_estimators=est, class_weight='balanced')
    model_rf.fit(features_train, target_train)
    predicted_valid_rf = model_rf.predict(features_valid)
    score_rf = f1_score(target_valid, predicted_valid_rf)
    if score_rf > best_f1_score_rf:
        best_est = est
        best_f1_score_rf = score_rf

print(f'Лучшее количество деревьев случайного леса: {best_est}, f1-score: {best_f1_score_rf:.3f}')

Лучшее количество деревьев случайного леса: 49, f1-score: 0.574


In [15]:
# Определение лучшей глубины случайного леса
best_depth_rf = 0
best_f1_score_rf = 0

for depth in range(1,50):
    model_rf = RandomForestClassifier(random_state=12345, n_estimators=27, max_depth=depth, class_weight='balanced')
    model_rf.fit(features_train, target_train)
    predicted_valid_rf = model_rf.predict(features_valid)
    score_rf = f1_score(target_valid, predicted_valid_rf)
    if score_rf > best_f1_score_rf:
        best_depth_rf = depth
        best_f1_score_rf = score_rf

print(f'Лучшая глубина случайного леса: {best_depth_rf}, f1-score: {best_f1_score_rf:.3f}')

Лучшая глубина случайного леса: 8, f1-score: 0.621


In [16]:
# Создание и обучение модели Решающее дерево
model_dt = DecisionTreeClassifier(random_state=12345, max_depth=5, class_weight='balanced')
сheck_adequacy_f1_roc('Решающее дерево', model_dt, features_valid, target_valid, features_train, target_train)

# Создание и обучение модели Случайный лес
model_rf = RandomForestClassifier(random_state=12345, n_estimators=27, max_depth=33, class_weight='balanced')
сheck_adequacy_f1_roc('Случайный лес', model_rf, features_valid, target_valid, features_train, target_train)


Решающее дерево
Доля правильных ответов: 0.8105, precision: 0.537, recall: 0.670, f1: 0.596, roc_auc: 0.831

Случайный лес
Доля правильных ответов: 0.8555, precision: 0.759, recall: 0.452, f1: 0.567, roc_auc: 0.828

Логистическая регрессия
Доля правильных ответов: 0.702, precision: 0.381, recall: 0.682, f1: 0.489, roc_auc: 0.764


In [17]:
# Рассмотрим увеличение выборки
# Создание функции upsample
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)

    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

In [18]:
# Определение лучшей глубины решающего дерева
best_depth_dt = 0
best_f1_score_dt = 0

for depth in range(1,30):
    model_dt = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_dt.fit(features_upsampled, target_upsampled)
    predicted_valid_dt = model_dt.predict(features_valid)
    score_dt = f1_score(target_valid, predicted_valid_dt)
    if score_dt > best_f1_score_dt:
        best_depth_dt = depth
        best_f1_score_dt = score_dt

print(f'Лучшая глубина решающего дерева: {best_depth_dt}, f1-score: {best_f1_score_dt:.3f}')

Лучшая глубина решающего дерева: 5, f1-score: 0.596


In [19]:
# Определение лучшего количества деревьев случайного леса
best_est = 0
best_f1_score_rf = 0

for est in range(1, 50):
    model_rf = RandomForestClassifier(random_state=12345, n_estimators=est)
    model_rf.fit(features_upsampled, target_upsampled)
    predicted_valid_rf = model_rf.predict(features_valid)
    score_rf = f1_score(target_valid, predicted_valid_rf)
    if score_rf > best_f1_score_rf:
        best_est = est
        best_f1_score_rf = score_rf

print(f'Лучшее количество деревьев случайного леса: {best_est}, f1-score: {best_f1_score_rf:.3f}')

Лучшее количество деревьев случайного леса: 31, f1-score: 0.603


In [20]:
# Определение лучшей глубины случайного леса
best_depth_rf = 0
best_f1_score_rf = 0

for depth in range(1,50):
    model_rf = RandomForestClassifier(random_state=12345, n_estimators=31, max_depth=depth)
    model_rf.fit(features_upsampled, target_upsampled)
    predicted_valid_rf = model_rf.predict(features_valid)
    score_rf = f1_score(target_valid, predicted_valid_rf)
    if score_rf > best_f1_score_rf:
        best_depth_rf = depth
        best_f1_score_rf = score_rf

print(f'Лучшая глубина случайного леса: {best_depth_rf}, f1-score: {best_f1_score_rf:.3f}')

Лучшая глубина случайного леса: 10, f1-score: 0.621


In [21]:
# Создание и обучение модели Решающее дерево
model_dt = DecisionTreeClassifier(random_state=12345, max_depth=5)
сheck_adequacy_f1_roc('Решающее дерево', model_dt, features_valid, target_valid, features_upsampled, target_upsampled)

# Создание и обучение модели Случайный лес
model_rf = RandomForestClassifier(random_state=12345, n_estimators=31, max_depth=10)
сheck_adequacy_f1_roc('Случайный лес', model_rf, features_valid, target_valid, features_upsampled, target_upsampled)


Решающее дерево
Доля правильных ответов: 0.8105, precision: 0.537, recall: 0.670, f1: 0.596, roc_auc: 0.831

Случайный лес
Доля правильных ответов: 0.8255, precision: 0.569, recall: 0.684, f1: 0.621, roc_auc: 0.850

Логистическая регрессия
Доля правильных ответов: 0.702, precision: 0.381, recall: 0.682, f1: 0.489, roc_auc: 0.764


In [22]:
# Рассмотрим уменьшение выборки
# Создание функции downsample
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat([features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat([target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    features_downsampled, target_downsampled = shuffle(features_downsampled, target_downsampled, random_state=12345)

    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.25)

In [23]:
# Определение лучшей глубины решающего дерева
best_depth_dt = 0
best_f1_score_dt = 0

for depth in range(1,30):
    model_dt = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_dt.fit(features_downsampled, target_downsampled)
    predicted_valid_dt = model_dt.predict(features_valid)
    score_dt = f1_score(target_valid, predicted_valid_dt)
    if score_dt > best_f1_score_dt:
        best_depth_dt = depth
        best_f1_score_dt = score_dt

print(f'Лучшая глубина решающего дерева: {best_depth_dt}, f1-score: {best_f1_score_dt:.3f}')

Лучшая глубина решающего дерева: 5, f1-score: 0.594


In [24]:
# Определение лучшего количества деревьев случайного леса
best_est = 0
best_f1_score_rf = 0

for est in range(1, 50):
    model_rf = RandomForestClassifier(random_state=12345, n_estimators=est)
    model_rf.fit(features_downsampled, target_downsampled)
    predicted_valid_rf = model_rf.predict(features_valid)
    score_rf = f1_score(target_valid, predicted_valid_rf)
    if score_rf > best_f1_score_rf:
        best_est = est
        best_f1_score_rf = score_rf

print(f'Лучшее количество деревьев случайного леса: {best_est}, f1-score: {best_f1_score_rf:.3f}')

Лучшее количество деревьев случайного леса: 16, f1-score: 0.597


In [25]:
# Определение лучшей глубины случайного леса
best_depth_rf = 0
best_f1_score_rf = 0

for depth in range(1,50):
    model_rf = RandomForestClassifier(random_state=12345, n_estimators=16, max_depth=depth)
    model_rf.fit(features_downsampled, target_downsampled)
    predicted_valid_rf = model_rf.predict(features_valid)
    score_rf = f1_score(target_valid, predicted_valid_rf)
    if score_rf > best_f1_score_rf:
        best_depth_rf = depth
        best_f1_score_rf = score_rf

print(f'Лучшая глубина случайного леса: {best_depth_rf}, f1-score: {best_f1_score_rf:.3f}')

Лучшая глубина случайного леса: 24, f1-score: 0.597


In [26]:
# Создание и обучение модели Решающее дерево
model_dt = DecisionTreeClassifier(random_state=12345, max_depth=5)
сheck_adequacy_f1_roc('Решающее дерево', model_dt, features_valid, target_valid, features_downsampled, target_downsampled)

# Создание и обучение модели Случайный лес
model_rf = RandomForestClassifier(random_state=12345, n_estimators=16, max_depth=24)
сheck_adequacy_f1_roc('Случайный лес', model_rf, features_valid, target_valid, features_downsampled, target_downsampled)


Решающее дерево
Доля правильных ответов: 0.8, precision: 0.516, recall: 0.701, f1: 0.594, roc_auc: 0.824

Случайный лес
Доля правильных ответов: 0.7945, precision: 0.506, recall: 0.730, f1: 0.597, roc_auc: 0.842

Логистическая регрессия
Доля правильных ответов: 0.7005, precision: 0.378, recall: 0.672, f1: 0.484, roc_auc: 0.762


<b>Вывод:</b>

При устранении дисбаланса в данных лучше всего показала техника "upsampling" для модели "Случайный лес" с количество деревьев 31 и с глубиной 10.

Применим ее для тестирования модели.

## Тестирование модели

In [27]:
# Тестирование модели Решающее дерево
model = RandomForestClassifier(random_state=12345, n_estimators=31, max_depth=10)
model.fit(features_upsampled, target_upsampled)
predicted = model.predict(features_test)
accuracy = accuracy_score(target_test, predicted)
precision = precision_score(target_test, predicted)
recall = recall_score(target_test, predicted)
f1 = f1_score(target_test, predicted)

probabilities_test = model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
auc_roc = roc_auc_score(target_test, probabilities_one_test)

print('Решающее дерево')
print(f'Доля правильных ответов: {accuracy}, precision: {precision:.3f}, recall: {recall:.3f}, f1: {f1:.3f}, roc_auc: {auc_roc:.3f}')

Решающее дерево
Доля правильных ответов: 0.818, precision: 0.560, recall: 0.655, f1: 0.603, roc_auc: 0.850


<div class="alert alert-block alert-warning">
<b>Комментарии студентки:</b>

После внесения корректировок по комментариям была достигнута цель: f1 > 0.59.

</div>

<b>Вывод:</b>

В ходе данной работы было выполнено следующее: 
- просмотрены и подготовлены данные к исследованию: 
 - пропущенные значения для Tenure заполнены медианой;
 - промасштабированны количественно-численные признаки;
 - сформированы выборки: обучающая, валидационная и тестовая;
- при обучении моделей подбирались гиперпараметры для:
 - решающее дерево;
 - cлучайный лес;
- при устранении дисбаланса в данных были применены техники:
 - взвешивание классов;
 - увеличение выборки - лучше всего показала данная техника для модели "Случайный лес" с f1: 0.621;
 - уменьшение выборки;
- при тестировании модели на тестовой выборке получили f1 - 0.603 (больше 0,59), при этом ROC-кривая оказалась - 0.85.